In [1]:
import torch
import numpy as np
import pandas as pd
import helper_functions as hf
from dataloader import JPXData
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataloader import default_collate
from torchvision import transforms


device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"

In [2]:
# Run this cell to create the new modified_data folder
# ~ 1 min runtime

# hf.concat_data()


In [3]:
# Run this cell to tokenize Section/Product and NewMarketSegmentColumns on stock_list.csv
# ~ 20 secs runtime

# hf.tokenize_stock_list()

In [4]:
batch_size = 1
data = JPXData(batch_size=batch_size)
trainloader = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=1)

In [5]:
a = data[0][0]
print(a)

tensor([ 0.2589, -0.4631,  0.3975,  ..., -0.6524, -0.6524, -0.6524])


In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(2000, 500),
            nn.ReLU(),
            nn.Linear(500, 500),
            nn.ReLU(),
            nn.Linear(500, 2000),
        )

    def forward(self, x):
        output = self.layer_stack(x)
        return output

In [7]:
net = Net().cuda()
print(net)

Net(
  (layer_stack): Sequential(
    (0): Linear(in_features=2000, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=500, out_features=2000, bias=True)
  )
)


In [8]:
criterion = nn.MSELoss().cuda()
optimizer = optim.SGD(net.parameters(), lr=1e-4, momentum=0.9)

In [9]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 200 == 199:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.10f}')
            running_loss = 0.0
            torch.save(net.state_dict(), f'saved_models/{epoch}_{i+1:5d}.pt')

print('Finished Training')

[1,   200] loss: 0.1010522186
[1,   400] loss: 0.1006247295
[1,   600] loss: 0.1006859123
[1,   800] loss: 0.1004682008
[1,  1000] loss: nan


KeyboardInterrupt: 